In [1]:
import streamlit as st
import regex as re
import faiss
import pickle, os 
from tqdm import tqdm
import pandas as pd
import numpy as np
import langchain
from langchain_community.document_loaders import UnstructuredURLLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.vectorstores.faiss import FAISS

import torch

# Check the available CUDA devices
available_devices = torch.cuda.device_count()
print(f"Available CUDA devices: {available_devices}")

for i in range(available_devices):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")

# Select the desired device
device_id = 0  # Change this to the desired device id
device = torch.device(f"cuda:{device_id}" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Available CUDA devices: 1
Device 0: NVIDIA GeForce RTX 3050 Laptop GPU
Using device: cuda:0


In [2]:
urls = [
    "https://www.thehindu.com/business/budget/budget-2024-live-updates-nirmala-sitharaman-union-budget-highlights/article68433121.ece"
]


url_loader  = UnstructuredURLLoader(urls=urls)

data = url_loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=100,
    
    length_function=len
)

In [4]:
docs = text_splitter.split_documents(data)[10:267]
docs = docs[10:267]

In [5]:
print(len(docs))
print(type(docs[0]))
print(docs[12])

257
<class 'langchain_core.documents.base.Document'>
page_content='A Flourish data visualization by Graphics Team 2

July 23, 2024 19:08

FM announces ₹10 lakh crore investment into scheme for urban housing' metadata={'source': 'https://www.thehindu.com/business/budget/budget-2024-live-updates-nirmala-sitharaman-union-budget-highlights/article68433121.ece'}


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv , dotenv_values
load_dotenv()


llm = ChatGoogleGenerativeAI(model='gemini-pro',google_api_key=os.getenv("GEMINI_API_KEY"))

d:\Generative AI Projects\WebArticle QA Bot\lcenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RetrieverQAwithSourceChain

In [7]:
# from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

model_kwargs = {'device': device}
encode_kwargs={"batch_size": 128}
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",model_kwargs=model_kwargs,encode_kwargs=encode_kwargs)

load INSTRUCTOR_Transformer
max_seq_length  512


d:\Generative AI Projects\WebArticle QA Bot\lcenv\lib\site-packages\sentence_transformers\models\Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_

In [8]:
vectorstore_db = FAISS.from_documents(docs,embedding=instructor_embeddings)

In [9]:
file_path = "vectorStorDB.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorstore_db,f)

In [14]:
with open('vectorStorDB.pkl', 'rb') as f:
    dest_object_name = pickle.load(f)

d:\Generative AI Projects\WebArticle QA Bot\lcenv\lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [10]:
from langchain.chains import RetrievalQAWithSourcesChain

chain = RetrievalQAWithSourcesChain.from_llm(llm=llm,retriever=vectorstore_db.as_retriever())

chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001BDD815E0A0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x000001BDDD054490>, default_metadata=())), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a lon

In [16]:
query = "what information about BSNL given in this document? Explain in details."

chain.invoke({"question":query},return_only_outputs=True)

{'answer': "- BSNL is receiving a significant investment of over ₹1 lakh crore for improving its technology and undergoing restructuring.\n\n- BSNL is set to receive ₹82,916 crore, which represents the majority of the allocated funds under the Telecom Ministry's ₹1.28 lakh crore allocation for telecom projects and public sector firms.\n\n",
 'sources': ''}